In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import numpy as np
import os
import cv2
import scipy.ndimage
import matplotlib.pyplot as plt

from scipy.ndimage import zoom

Mounted at /content/drive


In [ ]:
''' CROP OF BLACK PIXELS '''

def crop(img,num_pixel=0): # automatic crop of black pixels have been already performed in a previous script, that's why num_pixel=0

	return img[num_pixel:img.shape[0]-num_pixel,num_pixel:img.shape[1]-num_pixel]

In [ ]:
''' RESIZE OF THE IMAGE '''

def resize(img,new_dim=(200,250)): # (y,x)
	
	zoom0 = new_dim[0]/img.shape[0]
	zoom1 = new_dim[1]/img.shape[1]
	pix_resized = zoom(img,(zoom0,zoom1),order=3)
	
	return pix_resized

In [ ]:
''' INSERTING ALL THE SCANS IN A VECTOR '''

scans = []
path = '/content/drive/MyDrive/...' # add path of preprocessed scans
for r, d, f in os.walk(path):
  for file in f: 
    if file.endswith('.npy'):
      scans.append(os.path.join(r, file)) 

In [ ]:
''' RESIZING AND SAVING OF THE NEW RESIZED SCANS '''

i = 0
for i in range (len(scans)):
  scan = np.load(scans[i]) 
  scan = scan.astype(int)

  if scan.shape[0] > 250: # if n° slices >250, initial and final slices of the same scan are removed in equal number in order to center the volume of the lungs
    slices_to_delete = scan.shape[0] - 250
    print('Number of slices to delete for scan '+str(i+1)+': ',slices_to_delete)
    n2 = int(slices_to_delete // 2)
    if (slices_to_delete % 2) == 0: # if n° slices is even, I remove half from the beginning and half from the end
      ct_slices = scan[n2:-n2] 
    else: # if n° slices is odd, I remove half from the beginning and half+1 from the end
      n3 = n2+1
      ct_slices = scan[n2:-n3]
  else: # else, if the scan has <250 slices (or exactly 250), I do zero padding unltil reaching 250 slices for each scan
    slices_to_add = 250 - scan.shape[0]
    print('Number of slices to add for scan '+str(i+1)+': ',slices_to_add)
    ct_slices = np.pad(scan,((0,slices_to_add),(0,0),(0,0)),'constant') 
  print("Shape before standardizing the number of slices: "+str(scan.shape)) # z, y, x
  print("Shape after standardizing the number of slices: "+str(ct_slices.shape)) # z, y, x

  ct_resized = np.zeros((250,200,250)) #here you can change the dimension as you prefer
  for j in range(ct_slices.shape[0]): 
    ct_scan = crop(ct_slices[j])
    ct_resized[j] = resize(ct_scan)
  resized_scan = np.array(ct_resized)
  resized_scan = resized_scan.astype(np.int16)
  print("Shape before standardizing also the slice resolution: "+str(ct_slices.shape)) # z, y, x
  print("Shape after standardizing also the slice resolution: "+str(ct_resized.shape)) # z, y, x
  i+=1

  np.save("/content/drive/MyDrive/.../Standardized-scan-"+str(i),resized_scan) # add final path (the one where dimentionally-uniform scans will be saved)